In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv("sample_missing.csv", dtype={'PassengerId': int, 'Survived': int, 'Pclass': str, 'Name': str, 'Sex': str, 'Age': 'float16', 'SibSp': str, 'Parch': 'float16', 'Ticket': str, 'Fare': 'float16', 'Cabin': str, 'Embarked': str})
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0.0,A/5 21171,7.250000,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0.0,PC 17599,71.312500,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0.0,STON/O2. 3101282,7.925781,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0.0,113803,53.093750,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0.0,373450,8.046875,NaN,S


In [3]:
depvar = "Survived"
print("BEFORE")
N_Missing = pd.Series(train.isna().sum(), name="N_Missing")
Column_Type = pd.Series(train.dtypes, name="Column_Type")
N_Unique = pd.Series(train.nunique(), name="N_Unique")
treat = pd.concat([N_Missing, Column_Type, N_Unique], axis=1)
treat["Action"] = "No Action"
treat.loc[treat.Column_Type=='object', 'Action'] = 'Obj: dummies, dropfirst'
treat.loc[(treat.Column_Type=='object') & (treat.N_Missing>0), 'Action'] = 'Obj/missing: dummies, nodrop'
treat.loc[(treat.Column_Type!='object') & (treat.N_Missing>0), 'Action'] = 'Num/missing: impute odds'
print(treat)
print("-"*100)
dumcols = []
print("-------MISSING-------")
for col in train.columns[train.isna().sum()>0]:
    if np.issubdtype(train[col].dtype, np.number):
        this_nmX = pd.DataFrame(train[col][train[col].notna()])
        this_nmy = train[depvar][train[col].notna()]
        logreg = LogisticRegression()
        logreg.fit(this_nmX, this_nmy)
        impv = (math.log((train.loc[train[col].isna(), depvar].mean() + 10e-8)/(1-train.loc[train[col].isna(), depvar].mean() + 10e-8)) - logreg.intercept_[0])/logreg.coef_[0][0]
        impv = np.clip(impv, train[col].min(), train[col].max())
        train[col].fillna(value=impv, inplace=True)
        print(col, " imputed to value ", impv)
    else:
        if train[col].nunique() <= 20:
            dumcols.append(col)
            print(col, " encoded")
        else:
            print(f"{col} is object type and has {train[col].nunique()} distinct values, can't be imputed or encoded")
if len(dumcols) > 0:
    train = pd.get_dummies(train, columns=dumcols)
dumcols = []
print("-------NON MISSING-------")
for col in train.columns[(train.isna().sum()==0) & (train.dtypes=='object')]:
    if train[col].nunique() <= 20:
        dumcols.append(col)
        print(f"{col} encoded")
    else:
        print(f"{col} is object type and has {train[col].nunique()} distinct values, can't be imputed or encoded")
if len(dumcols) > 0:
    train = pd.get_dummies(train, columns=dumcols, drop_first=True)
print("-"*100)
print("AFTER")
N_Missing = pd.Series(train.isna().sum(), name="N_Missing")
Column_Type = pd.Series(train.dtypes, name="Column_Type")
N_Unique = pd.Series(train.nunique(), name="N_Unique")
treat = pd.concat([N_Missing, Column_Type, N_Unique], axis=1)
treat['Action'] = 'No Action'
treat.loc[treat.Column_Type=='object', 'Action'] = 'Obj: dummies, dropfirst'
treat.loc[(treat.Column_Type=='object') & (treat.N_Missing>0), 'Action'] = 'Obj/missing: dummies, nodrop'
treat.loc[(treat.Column_Type!='object') & (treat.N_Missing>0), 'Action'] = 'Num/missing: impute odds'
print(treat)
print("-------Sample of columns with pending action-------")
for col in train.columns[treat.Action!='No Action']:
    print(col)
    print(train.loc[train[col].notna(), col].head().values)

BEFORE
             N_Missing Column_Type  N_Unique                        Action
PassengerId          0       int32       891                     No Action
Survived             0       int32         2                     No Action
Pclass               7      object         3  Obj/missing: dummies, nodrop
Name                 0      object       891       Obj: dummies, dropfirst
Sex                  0      object         2       Obj: dummies, dropfirst
Age                177     float16        88      Num/missing: impute odds
SibSp                5      object         7  Obj/missing: dummies, nodrop
Parch                4     float16         7      Num/missing: impute odds
Ticket               0      object       681       Obj: dummies, dropfirst
Fare                 0     float16       242                     No Action
Cabin              687      object       147  Obj/missing: dummies, nodrop
Embarked             2      object         3  Obj/missing: dummies, nodrop
------------------